In [1]:
%%capture
!pip install langchain langchain_core langchain_groq langchain_community langgraph gradio
!pip install gradio pandas uuid chromadb langchain langchain-community langchain-core pymupdf openai-whisper
!pip install gTTS

In [21]:
import gradio as gr
import fitz  # PyMuPDF
import mimetypes
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WebBaseLoader
from langchain_groq import ChatGroq
from gtts import gTTS
#import pyttsx3 for run it in locally
import os
import uuid

In [22]:
llm = ChatGroq(
    temperature=0,
    groq_api_key="Your api",
    model_name="llama-3.3-70b-versatile"
)

In [24]:
def load_cv_text(cv_file):
    file_type, _ = mimetypes.guess_type(cv_file.name)
    if file_type == 'application/pdf':
        try:
            doc = fitz.open(cv_file.name)
            text = "\n".join(page.get_text() for page in doc)
            doc.close()
            return text
        except Exception as e:
            return f"Failed to read PDF: {e}"
    else:
        try:
            return cv_file.read().decode("utf-8")
        except Exception:
            return "Unsupported file format or decoding failed."

def analyze_cv_against_job(cv_text, job_text):
    # Detailed, friendly prompt for the LLM
    prompt = PromptTemplate.from_template("""
Hey! You're a super friendly and helpful CV coach.

Please compare the CV below with the job description and give me:

1. A score out of 100 that shows how well the CV fits the job.
2. Very detailed, easy-to-understand, and casual feedback on what could be improved.
   - Break down feedback clearly by areas like skills, experience, formatting, or anything missing.
   - Explain every fault or gap in a positive and encouraging way, so the candidate feels motivated to improve.
   - Use simple language that anyone can understand.
3. If the CV is great (score 85+), also add a warm "Good job! Your CV looks solid."

Return the result strictly as JSON:

{{
  "score": int,
  "feedback": string
}}

### CV:
{cv}

### JOB DESCRIPTION:
{job}
""")

    parser = JsonOutputParser()
    chain = prompt | llm | RunnableLambda(lambda x: x.content.strip())

    try:
        raw = chain.invoke({"cv": cv_text, "job": job_text})
        result = parser.parse(raw)
        return result["score"], result["feedback"]
    except Exception as e:
        return 0, f"Oops, something went wrong: {e}"

def text_to_speech(feedback, voice_gender):
    lang_code = "en-IN"


    tld = "com" if voice_gender == "Man" else "co.in"

    try:
        tts = gTTS(text=feedback, lang=lang_code, tld=tld)
        path = f"/tmp/feedback_{uuid.uuid4().hex}.mp3"
        tts.save(path)
        return path
    except Exception as e:
        print("Speech generation failed:", e)
        return None

def main(cv_file, job_text_or_url, voice_gender):
    cv_text = load_cv_text(cv_file)
    if job_text_or_url.startswith("http"):
        loader = WebBaseLoader(job_text_or_url)
        job_text = loader.load().pop().page_content
    else:
        job_text = job_text_or_url

    score, feedback = analyze_cv_against_job(cv_text, job_text)
    audio = text_to_speech(feedback, voice_gender)
    report = f"Score: {score}/100\n\nFeedback:\n{feedback}"
    return report, audio

ui = gr.Interface(
    fn=main,
    inputs=[
        gr.File(label="Upload CV (PDF or TXT)"),
        gr.Textbox(label="Job Description or URL", lines=10, placeholder="Paste job description text or URL here"),
        gr.Radio(["Voice1", "Voice2"], label="Select Voice for Speech Feedback", value="Woman")
    ],
    outputs=[
        gr.Textbox(label="Feedback Report"),
        gr.Audio(label="Speech Feedback", type="filepath")
    ],
    title="CV Analyzer & Friendly Feedback Generator",
    description="Upload your CV and job description. Get scored feedback and friendly, detailed improvement tips with selectable voices.",
    allow_flagging="never"
)

ui.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a930a927fdf8ea011.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
